In [1]:
import matplotlib.pyplot as plt
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="4"

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, LabelBinarizer, Normalizer
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import pickle
import numpy as np
import pandas as pd
from random import choice

from annsa.template_sampling import *

In [2]:
from hyperparameter_models import make_conv1d_model as make_model

In [3]:
import tensorflow.contrib.eager as tfe

In [4]:
tf.enable_eager_execution()

#### Import model, training function 

In [5]:
from annsa.model_classes import (cnn1d_model_features,
                                 CNN1D,
                                 generate_random_cnn1d_architecture,
                                 save_model,
                                 train_earlystop)

## Load testing dataset 

In [6]:
dataset = np.load('../dataset_generation/testing_dataset_full_200keV_100.npy')

In [7]:
all_spectra = np.add(dataset.item()['sources'], dataset.item()['backgrounds'])
all_keys = dataset.item()['keys']

mlb=LabelBinarizer()

all_keys_binarized = mlb.fit_transform(all_keys)

## Train network

### Define hyperparameters

In [8]:
number_hyperparameters_to_search = 256
earlystop_errors_test = []
model_id='CNN-kfoldsfull-updated-5'

### Search hyperparameters

In [ ]:
def save_features(model_features,
                  model_id,
                  hyperparameter_index):
    with open('./hyperparameter-search-results/' + model_id + '_' +
              str(hyperparameter_index), 'wb+') as f:
        pickle.dump(model_features,f)
    return None

skf = StratifiedKFold(n_splits=5, random_state=5)
testing_errors = []
all_kf_errors = []

for network_id in range(number_hyperparameters_to_search):
    print(network_id)
    model, model_features = make_model(all_keys_binarized)
    save_features(model_features,
                  model_id,
                  network_id)
    
    k_folds_errors = []
    for train_index, test_index in skf.split(all_spectra, all_keys):
        # reset model on each iteration
        model = CNN1D(model_features)
        optimizer = tf.train.AdamOptimizer(model_features.learining_rate)

        costfunction_errors_tmp, earlystop_errors_tmp = train_earlystop(
                training_data=all_spectra[train_index],
                training_keys=all_keys_binarized[train_index],
                testing_data=all_spectra[test_index],
                testing_keys=all_keys_binarized[test_index],
                model=model,
                optimizer=optimizer,
                num_epochs=200,
                obj_cost=model.cross_entropy,
                earlystop_cost_fn=model.f1_error,
                earlystop_patience=10,
                not_learning_patience=10,
                not_learning_threshold=0.9,
                verbose=True,
                fit_batch_verbose=10,
                data_augmentation=model.default_data_augmentation)
        k_folds_errors.append(earlystop_errors_tmp)
        all_kf_errors.append(earlystop_errors_tmp)

    testing_errors.append(np.average(k_folds_errors))
    np.save('./final-models/final_test_errors_'+model_id, testing_errors)
    np.save('./final-models/final_kf_errors_'+model_id, all_kf_errors)
    # model.save_weights('./final-models/'+model_id+'_checkpoint_'+str(network_id))
    network_id += 1 

0
Epoch 10: CostFunc loss: 0.00 3.43, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.68 Early stopfctn: 3.68
Epoch 10: CostFunc loss: 0.00 3.43, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.69 Early stopfctn: 3.69
Epoch 10: CostFunc loss: 0.00 3.41, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.52 Early stopfctn: 3.52
Epoch 10: CostFunc loss: 0.00 3.40, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.41 Early stopfctn: 3.41
Epoch 10: CostFunc loss: 0.00 3.48, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.87 Early stopfctn: 3.87
1
Epoch 10: CostFunc loss: 0.00 3.93, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.47 Early stopfctn: 4.47
Epoch 10: CostFunc loss: 0.00 3.84, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 4.38 Early stopfctn: 4.38
Epoch 10: CostFunc loss: 0.00 3.80, EarlyStop loss: 0.00 0.97
Test error at ear

Test error at early stop: Objectives fctn: 3.40 Early stopfctn: 3.40
Epoch 10: CostFunc loss: 0.00 3.40, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.40 Early stopfctn: 3.40
Epoch 10: CostFunc loss: 0.00 3.40, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.40 Early stopfctn: 3.40
Epoch 10: CostFunc loss: 0.00 3.40, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.40 Early stopfctn: 3.40
Epoch 10: CostFunc loss: 0.00 3.40, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.40 Early stopfctn: 3.40
12
Epoch 10: CostFunc loss: 0.00 3.42, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.46 Early stopfctn: 3.46
Epoch 10: CostFunc loss: 0.00 3.43, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.47 Early stopfctn: 3.47
Epoch 10: CostFunc loss: 0.00 3.41, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.43 Early stopfctn: 3.43
Epoch 10: C

Epoch 10: CostFunc loss: 0.00 3.42, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.42 Early stopfctn: 3.42
Epoch 10: CostFunc loss: 0.00 3.42, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.41 Early stopfctn: 3.41
Epoch 10: CostFunc loss: 0.00 3.41, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.42 Early stopfctn: 3.42
Epoch 10: CostFunc loss: 0.00 3.42, EarlyStop loss: 0.00 0.97
Test error at early stop: Objectives fctn: 3.41 Early stopfctn: 3.41
24
Epoch 10: CostFunc loss: 0.00 3.36, EarlyStop loss: 0.00 0.94
Test error at early stop: Objectives fctn: 3.77 Early stopfctn: 3.77
Epoch 10: CostFunc loss: 0.00 3.24, EarlyStop loss: 0.00 0.89
Epoch 20: CostFunc loss: 0.00 2.09, EarlyStop loss: 0.00 0.64
Epoch 30: CostFunc loss: 0.00 1.58, EarlyStop loss: 0.00 0.48
Test error at early stop: Objectives fctn: 1.54 Early stopfctn: 1.54
Epoch 10: CostFunc loss: 0.00 3.48, EarlyStop loss: 0.00 0.96
Test error at early stop: